# Pinecone

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
from pinecone import Pinecone

pc = Pinecone() #automaticall loads key from .env
# pc = Pinecone(api_key="PINECONE_API_KEY") # if key not entereed in .env
pc.list_indexes()

{'indexes': []}